In [5]:
import pandas as pd
from lxml import etree
import Capstone_functions as cf
import os
import sql_functions as sf

#### Global variables

In [6]:
parser = etree.XMLParser(recover=True)
engine = sf.get_engine()
schema = "BGG_Data"
xtree = etree.parse("data/old_API_id_260200.xml", parser= parser).getroot()
subnodes = [
    'boardgamecategory',
    'boardgamesubdomain',
    'boardgamemechanic',
    'boardgamefamily',
    'boardgameexpansion',
    'boardgamehonor',
    'boardgamedesigner',
    'boardgameartist',
    'boardgamepublisher',
    'boardgamepodcastepisode',
    'boardgameimplementation',
    'videogamebg',
    'statistics',
    'marketplacelistings'
]

#### Build df_main

In [7]:
df_main = pd.DataFrame()
directory = os.fsencode("data/")

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".xml"): 
        file_path = os.path.join(str(directory)[2:].replace("'",""), filename)
        xtree_temp = etree.parse(file_path, parser= parser).getroot()
        df_main = pd.concat([df_main,cf.df_main(xtree_temp)])


In [8]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 333317 entries, 0 to 185
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             333317 non-null  object
 1   description    333317 non-null  object
 2   yearpublished  137153 non-null  object
 3   min_players    137153 non-null  object
 4   max_players    137153 non-null  object
 5   playtime       137153 non-null  object
 6   min_playtime   137153 non-null  object
 7   max_playtime   137153 non-null  object
 8   min_age        137153 non-null  object
dtypes: object(9)
memory usage: 25.4+ MB


#### Loop over subnodes and add dataframes to dictionary

In [9]:
df_dict = dict()
directory = os.fsencode("data/")
for entrypoint in subnodes:
    df_temp = pd.DataFrame()
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".xml"): 
            file_path = os.path.join(str(directory)[2:].replace("'",""), filename)
            xtree_temp = etree.parse(file_path, parser= parser).getroot()
            df_temp = pd.concat([df_temp,cf.df_subnodes(xtree_temp, entrypoint=entrypoint)])
    df_dict[entrypoint] = df_temp

In [ ]:
for node in subnodes:
    print(node)
    df_dict[node].info()

In [11]:
sf.build_table(engine=engine,table_name="df_main_dirty",dataframe=df_main,schema=schema)

The df_main_dirty table was imported successfully.


In [ ]:
for entrypoint in subnodes:
    sf.build_table(engine=engine,table_name=entrypoint,dataframe=df_dict[entrypoint],schema=schema)